<a href="https://colab.research.google.com/github/Gilgamesh60/intern_assignment/blob/main/assignments_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The problem statement here is to extract details from the given pdf file.
These details include information based on both textual part and symbol images in the pdf document file.

# Downloading modules

**We will use pytesseract to detect text and pdf2image module to convert pdf to images**

In [ ]:
!pip3 install pytesseract

In [ ]:
!apt install tesseract-ocr

In [ ]:
!pip install pdf2image

In [ ]:
!sudo apt-get install poppler-utils

# Importing libraries

In [8]:
import pytesseract
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import os
from pdf2image import convert_from_path

# Image Processing

In [7]:
def image_smoothening(img):
  # thresholding with less noise 
  ret1, th1 = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
  blur = cv2.GaussianBlur(th1, (5, 5), 0)
  ret2, th2 = cv2.threshold(th1, 0, 255, cv2.THRESH_BINARY)
  return th2

def image_process(image):
  img=np.asarray(image)
  gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  ret,thresh1 = cv2.threshold(gray,170, 255, cv2.THRESH_BINARY)
  kernel = np.ones((1, 1), np.uint8)
  thresh = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)
  thresh = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, kernel)
  smooth_img = image_smoothening(gray)
  final_img = cv2.bitwise_or(smooth_img, thresh)
  return final_img

# Text data extraction

In [9]:
def info_finder(PDF_file_path):
  Refs=[]
  Device_name=[]
  Lots=[]
  qty=[]
  images = convert_from_path(PDF_file_path)
  for image in images :
    img=image_process(image)
    text=pytesseract.image_to_string(img)
    text=text.replace('\n',' ')
    Refs.append(text.split('REF ')[1].split(' LOT')[0])
    Device_name.append(text.split('Device Name: ')[1].split('  REF')[0])
    Lots.append(int(text.split('LOT: ')[1].split(' ')[0]))
    img=np.asarray(image)
    image=img[0:390,735:1471]
    text=pytesseract.image_to_string(image)
    qty.append(int(text.split('Qty: ')[1].split('\n')[0]))
  return Refs,Device_name,Lots,qty

In [214]:
Refs=info_finder('/content/Label.pdf')[0]
Device_name=info_finder('/content/Label.pdf')[1]
Lots=info_finder('/content/Label.pdf')[2]
qty=info_finder('/content/Label.pdf')[3]

## Symbol sequence detection

In [205]:
def sequence_finder(image):
  coords=[]
  image=cv2.resize(image,(1471,780))
  image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  for i in range(9):
    template=cv2.imread(f'/content/drive/MyDrive/symbols/{i+1}.png',0)
    template=cv2.resize(template,(191,76))
    w, h = template.shape[::-1]
    res = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    threshold = 0.6
    loc = np.where(res >= threshold)
    pts0=[]
    pts1=[]
    for pt in zip(*loc[::-1]):
       pts0.append(pt[0])
       pts1.append(pt[1])
    if len(pts0)!=0:
        coords.append((min(pts0),min(pts1),i+1))
        #y_coords.append((min(pts1),i+1))
    else:
          continue
    b=np.array(coords)
    #b= b[b[:, 1].argsort(kind='mergesort')]
    b2= b[b[:, 1].argsort(kind='mergesort')]
    if len(b)>1:
      if b2[-1][1]-b2[0][1]>10 and b2[-1][1]-b2[-2][1]<2:
          b1=b2[b2[0:, 0].argsort(kind='mergesort')]
          b1=np.vstack([b2[0],b1])
          b1=b1[:-1]
      elif b2[-1][1]-b2[0][1]>10 and b2[-1][1]-b2[-2][1]>10:
        b1=b2[b2[:-1, 0].argsort(kind='mergesort')]
        b1 = np.vstack([b1, b2[-1]])
      else:
        b1=b2[b2[:, 0].argsort(kind='mergesort')]
    else:
      b1=b;
    s=[]
    for i in range(len(b1)):
      s.append(b1[i][2])
    def convert(list):
     
      # Converting integer list to string list
      s = [str(i) for i in list]
     
      # Join list items using join()
      res = int("".join(s))
      return(res)
  return convert(s)

In [221]:
def sequencebuilder(PDF_file_path):
  symbol_ids=[]
  images = convert_from_path(PDF_file_path)
  for image in images:
    img=np.asarray(image)
    symbol_ids.append(sequence_finder(img))
  return symbol_ids

In [222]:
symbol_ids=sequencebuilder('/content/Label.pdf')

In [223]:
len(symbol_ids)

9

# Dataframe building

In [224]:
data=[]
for i in range(len(symbol_ids)):
  data.append((Device_name[i],Refs[i],Lots[i],qty[i],symbol_ids[i]))
data=np.array(data)

In [227]:
index_values = [1,2,3,4,5,6,7,8,9]
df = pd.DataFrame(data = data,  index = index_values, 
                  columns = ['Device Name','REF','LOT','Qty','Symbols'])
df

,Device Name,REF,LOT,Qty,Symbols
1,Bi-Pap Machine,NML903110,34669,1,3578
2,Blood Warmer,NML903090,34641,1,1
3,C-Pap Machine,NML903105,34662,1,15789
4,ECG Machine,NML903060,34690,9,2578
5,HFNC Machine,NML903095,34648,5,1289
6,Infusion Pump,NML903065,34697,10,2589
7,NIBP Monitor,NML903050,34676,5,4127
8,Oxygen Concentrator,NML903100,34655,2,43567
9,Pulse Oximeter,NML903055,34683,4,2369


In [228]:
df.to_csv('outputs.csv')
#symbol order is wrong for some reason